In [2]:
!pip install transformers 

     |████████████████████████████████| 3.5 MB 14.1 MB/s 
     |████████████████████████████████| 6.8 MB 51.9 MB/s 
     |████████████████████████████████| 895 kB 53.0 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !pip install sentencepiece

In [3]:
import pandas as pd

In [4]:
from google.colab import files
uploaded = files.upload()

Saving spam_data.csv to spam_data.csv


In [5]:
import io
data = pd.read_csv(io.BytesIO(uploaded['spam_data.csv']))
data 

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
#sultan/BioM-ELECTRA-Base-Discriminator
#does not require taking mean from the output
#output is token for each word

from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("sultan/BioM-ELECTRA-Base-Discriminator")

model = AutoModelForPreTraining.from_pretrained("sultan/BioM-ELECTRA-Base-Discriminator")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/220k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [32]:
example_input = ["I will watch movie tonight","Hi I am fine"]
print("\n")
print("TOKENS.............")
print([tokenizer.tokenize(i) for i in example_input]) 
print("\n")
print("TOKEN ENCODER...............")
example_output = tokenizer(example_input,padding='max_length', max_length = 15, 
                       truncation=True, return_tensors="pt")

print(example_output) 



TOKENS.............
[['i', 'will', 'wat', '##ch', 'mov', '##ie', 'ton', '##ight'], ['hi', 'i', 'am', 'fine']]


TOKEN ENCODER...............
{'input_ids': tensor([[    2,    50,  2982,  8713,  1831,  9282,  3248, 15045,  2001,     3,
             0,     0,     0,     0,     0],
        [    2, 11245,    50,  1925,  8053,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [33]:
print(example_output["input_ids"].shape)
print(example_output["token_type_ids"].shape)
print(example_output["attention_mask"].shape) 

torch.Size([2, 15])
torch.Size([2, 15])
torch.Size([2, 15])


In [34]:
out = model(example_output["input_ids"],example_output["attention_mask"])
print(out)

ElectraForPreTrainingOutput(loss=None, logits=tensor([[-7.7169, -3.3260, -3.5448, -8.1897, -7.8256, -7.4773, -6.8825, -2.1196,
         -1.2574, -7.7169, -0.9727, -0.1492, -0.1814, -0.0261, -0.0497],
        [-5.6776,  0.8574, -2.6181, -3.2283, -2.8699, -5.6776, -2.1290, -1.9455,
         -1.9007, -1.9670, -1.8714, -1.8152, -1.7978, -1.8397, -1.8363]],
       grad_fn=<SqueezeBackward1>), hidden_states=None, attentions=None)


In [35]:
print(out[0].shape)

torch.Size([2, 15])


## Specify the below paramaters based on the output from the transformer model

In [46]:

#whether you need to take the mean of the output or not- depends upon the transformer model output
take_mean = False 
#which output are you taking from all the outputs you got from the transformer model- if it is the fisrt output keep this as 0 itslef
return_index = 0
#maximum length of the output sequence
maxlen = 256
#output dimension to be returned from the transformer model
output_dim = maxlen
#batch size
bs = 4
#number of classes
classes = 2
#label dictionary
labels = {"spam":0,"ham":1}
label_column_name = "Category"
text_column_name = "Message"

# Dataset

In [47]:
import torch
import numpy as np


In [48]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self,df):
        self.labels = [labels[label] for label in df[label_column_name]]
        self.texts = [tokenizer(text, padding='max_length', max_length = maxlen, truncation=True, return_tensors="pt") 
                      for text in df[text_column_name]]
        
    def classes(self):
        return self.labels
        
    def __len__(self):
        return len(self.labels)
        
    def get_batch_labels(self,idx):
        return self.labels[idx] 
        
    def get_batch_text(self,idx):
        return self.texts[idx]
        
    def __getitem__(self,idx):
        batch_x = self.get_batch_text(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_x,batch_y
        

In [49]:
np.random.seed(0)
df_train, df_val, df_test = np.split(data.sample(frac=1, random_state=42), 
                                     [int(.8*len(data)), int(.9*len(data))])
print(df_train.shape, df_val.shape, df_test.shape)

(4457, 2) (557, 2) (558, 2)


# Model building

In [50]:
from torch import nn

class Classifier(nn.Module):
    def __init__(self, tf_model,output_dim,dropout=0.5):
        super(Classifier, self).__init__()
        self.bert = tf_model
        self.dropout = nn.Dropout(dropout) 
        self.linear = nn.Linear(output_dim,classes)
        self.relu = nn.ReLU()
        
    def forward(self,input_id, mask,return_mean):
        pooled_out = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)[return_index]
        if return_mean:
          pooled_out = torch.mean(pooled_out,1)
        out = self.dropout(pooled_out)
        
        out = self.linear(out)
        out = self.relu(out)
        
        return out
    

# Training loop

In [51]:
from torch.optim import Adam
from tqdm import tqdm


In [52]:
def train(model, train_data, val_data, learning_rate, epochs):
    train1,val1 = Dataset(train_data), Dataset(val_data)
    
    train_dataloader = torch.utils.data.DataLoader(train1, batch_size=bs, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val1, batch_size=bs, shuffle=True)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    for epoch in range(epochs):
        total_train_acc = 0
        total_train_loss = 0
        
        for text,label in tqdm(train_dataloader):
            label = label.to(device)          
            
            input_id = torch.squeeze(text["input_ids"], 1)
            mask = torch.squeeze(text["attention_mask"],1)
            input_id = input_id.to(device)
            mask = mask.to(device)
                    
            output = model(input_id,mask,return_mean = take_mean)
            # print(output.shape)
#             print(label.shape)
            batch_loss = criterion(output, label)
            total_train_loss+=batch_loss.item()
            acc= (output.argmax(dim=1)==label).sum().item()
            total_train_acc+=acc
            
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            
        total_val_acc = 0
        total_val_loss = 0
        with torch.no_grad():
            for val_text,val_label in tqdm(val_dataloader):
                val_label = val_label.to(device)
                input_id = torch.squeeze(val_text["input_ids"], 1)
                mask = torch.squeeze(val_text["attention_mask"],1)

                input_id = input_id.to(device)
                mask = mask.to(device)
                output = model(input_id,mask,return_mean = take_mean)
                batch_loss = criterion(output, val_label)
                total_val_loss+=batch_loss.item()
                acc= (output.argmax(dim=1)==val_label).sum().item()
                total_val_acc+=acc
            
        print(
                f'Epochs: {epoch + 1} | Train Loss: {total_train_loss / len(train_data): .3f} | Train Accuracy: {total_train_acc / len(train_data): .3f} | Val Loss: {total_val_loss / len(val_data): .3f} | Val Accuracy: {total_val_acc / len(val_data): .3f}')
                  
        
            

In [53]:
train1,val1 = Dataset(df_train), Dataset(df_val)
criterion = nn.CrossEntropyLoss()
val_dataloader = torch.utils.data.DataLoader(val1, batch_size=bs, shuffle=True)
for val_text,val_label in tqdm(val_dataloader):
                
        input_id = torch.squeeze(val_text["input_ids"], 1)
        mask = torch.squeeze(val_text["attention_mask"],1)
        pooled_out = model(input_ids= input_id, attention_mask=mask,return_dict=False)[return_index]
        if take_mean:
          pooled_out = torch.mean(pooled_out,1)
        out = nn.Dropout(0.2)(pooled_out)
        out = nn.Linear(output_dim,classes)(out)
        out = nn.ReLU()(out)
        print(out.shape)
        batch_loss = criterion(out, val_label)
        print(batch_loss)
        break 

  0%|          | 0/140 [00:02<?, ?it/s]

torch.Size([4, 2])
tensor(0.6543, grad_fn=<NllLossBackward0>)


In [54]:
EPOCHS = 1
model1 = Classifier(model,output_dim)
LR = 1e-6
train(model1, df_train, df_val, LR, EPOCHS)  

  1%|          | 11/1115 [00:02<04:13,  4.35it/s]


KeyboardInterrupt: ignored

## KFold Cross validation

In [55]:
no_of_splits = 5

In [56]:
#cross validation
from sklearn.model_selection import KFold

def train_cv(model, data, learning_rate, epochs):
    splits= KFold(n_splits = no_of_splits, shuffle=True)
    best_acc = 0.0
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data)))): 
      print('------------fold no---------{}----------------------'.format(fold))
      train1 = Dataset(data.iloc[train_idx])
      val1 = Dataset(data.iloc[val_idx])
      train_dataloader = torch.utils.data.DataLoader(train1, batch_size=bs, shuffle=True)
      val_dataloader = torch.utils.data.DataLoader(val1, batch_size=bs, shuffle=True)
    
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      
      criterion = nn.CrossEntropyLoss()
      optimizer = Adam(model.parameters(), lr= learning_rate)
      
      model = model.to(device)
      criterion = criterion.to(device)

    
      for epoch in range(epochs):
          total_train_acc = 0
          total_train_loss = 0
          
          for text,label in tqdm(train_dataloader):
              label = label.to(device)          
              
              input_id = torch.squeeze(text["input_ids"], 1)
              mask = torch.squeeze(text["attention_mask"],1)
              input_id = input_id.to(device)
              mask = mask.to(device)
                      
              output = model(input_id,mask,return_mean= take_mean)
              # print(output.shape)
  #             print(label.shape)
              batch_loss = criterion(output, label)
              total_train_loss+=batch_loss.item()
              acc= (output.argmax(dim=1)==label).sum().item()
              total_train_acc+=acc
              
              model.zero_grad()
              batch_loss.backward()
              optimizer.step()
              
          total_val_acc = 0
          total_val_loss = 0
          with torch.no_grad():
              for val_text,val_label in tqdm(val_dataloader):
                  val_label = val_label.to(device)
                  input_id = torch.squeeze(val_text["input_ids"], 1)
                  mask = torch.squeeze(val_text["attention_mask"],1)

                  input_id = input_id.to(device)
                  mask = mask.to(device)
                  output = model(input_id,mask, return_mean= take_mean)
                  batch_loss = criterion(output, val_label)
                  total_val_loss+=batch_loss.item()
                  acc= (output.argmax(dim=1)==val_label).sum().item()
                  total_val_acc+=acc
              
          print(
                  f'Epochs: {epoch + 1} | Train Loss: {total_train_loss / len(train_idx): .3f} | Train Accuracy: {total_train_acc / len(train_idx): .3f} | Val Loss: {total_val_loss / len(val_idx): .3f} | Val Accuracy: {total_val_acc / len(val_idx): .3f}')
                  
      if total_val_acc/len(val_idx)>best_acc:
        best_acc = total_val_acc/len(val_idx)
        print("The best accuracy is--->", best_acc)
      else:
        print("accuracy has not improved from the previous fold")
      print(".........................................................................................................")

            

In [57]:
EPOCHS = 1
model1 = Classifier(model,output_dim)
LR = 1e-6
train_cv(model1, data, LR, EPOCHS) 

------------fold no---------0----------------------


  1%|          | 11/1115 [00:02<03:43,  4.93it/s]


KeyboardInterrupt: ignored

In [58]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=bs)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].squeeze(1).to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask,return_mean = take_mean)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [59]:
evaluate(model1, df_test)

Test Accuracy:  0.606


# Misc.

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'I will watch Memento tonight'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [ ]:
from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-cased')
o=bert(bert_input["input_ids"],bert_input["attention_mask"],return_dict=False)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
o[0].shape
print(torch.mean(o[0],1).shape)
torch.mean(o[0],1)

torch.Size([1, 768])


tensor([[ 3.5039e-01, -1.9487e-01,  1.0387e-01,  1.7135e-01,  3.0013e-01,
          9.3626e-02,  1.7778e-01,  7.3870e-02,  1.4471e-01, -5.8956e-02,
         -1.6891e-01,  2.9234e-01, -3.1623e-01,  3.6891e-01, -2.5948e-01,
         -1.1955e-01, -1.8508e-02, -8.9950e-02,  2.9625e-02, -2.0191e-01,
         -5.4164e-02,  6.2005e-02, -3.1030e-01, -4.7955e-02,  2.4356e-01,
         -2.1378e-01,  6.4915e-01,  7.0313e-01,  5.6117e-02,  1.7772e-01,
          4.2362e-02, -7.8822e-02,  1.2682e-01, -2.8575e-03, -1.6492e-01,
         -1.3824e-01, -1.9517e-01,  1.1798e-01, -2.6482e-01, -1.2428e-01,
         -2.9498e-01,  4.2889e-02,  2.7335e-01, -3.6402e-01,  4.9475e-02,
         -3.5574e-01,  1.2514e-01, -1.0856e-01, -2.2748e-01,  1.3191e-01,
          2.8182e-01, -2.2908e-01, -1.4320e-01, -3.8938e-02,  2.4720e-01,
         -6.9256e-02, -8.4726e-02,  2.9137e-02, -2.4527e-01,  3.3444e-01,
         -6.1131e-02,  3.2142e-01,  2.4662e-01,  1.1562e-01, -3.6122e-01,
          1.9660e-01,  1.3246e-01, -1.

In [ ]:
print(o[1])

tensor([[-0.7759,  0.5508,  1.0000, -0.9964,  0.9606,  0.9481,  0.9916, -0.9851,
         -0.9852, -0.6111,  0.9912,  0.9993, -0.9972, -0.9999,  0.7664, -0.9904,
          0.9933, -0.6442, -1.0000, -0.6149, -0.3549, -1.0000,  0.3385,  0.9545,
          0.9848,  0.0870,  0.9949,  1.0000,  0.8574,  0.0182,  0.3935, -0.9947,
          0.8052, -0.9996,  0.3009, -0.0659,  0.4360, -0.4595,  0.4401, -0.8517,
         -0.7425, -0.4531,  0.5130, -0.6632,  0.9282,  0.2589,  0.3070, -0.0266,
         -0.1987,  1.0000, -0.9823,  0.9999, -0.9900,  0.9993,  0.9978,  0.3938,
          0.9970,  0.2153, -0.9965,  0.4546,  0.9678,  0.1873,  0.9545, -0.3831,
         -0.0789, -0.5588, -0.7788,  0.4396, -0.4059,  0.4095,  0.5292,  0.4842,
          0.9940, -0.9471, -0.1695, -0.9478,  0.2050, -1.0000,  0.9759,  1.0000,
          0.6371, -0.9999,  0.9973, -0.3195, -0.6162, -0.0515, -0.9973, -0.9998,
          0.2102, -0.6552,  0.9261, -0.9936,  0.2623, -0.8178,  1.0000, -0.9664,
         -0.3420,  0.4940,  

In [ ]:
loss = nn.CrossEntropyLoss()
>>> input = torch.randn(3, 5, requires_grad=True)
>>> target = torch.empty(3, dtype=torch.long).random_(5)
>>> output = loss(input, target)

In [ ]:
print(input.shape)
print(target.shape)

torch.Size([3, 5])
torch.Size([3])


In [ ]:
print(input)

tensor([[ 1.9612, -0.4676,  0.5422, -0.9910,  0.6087],
        [-0.1090, -1.5409, -2.7575,  0.8405, -0.1283],
        [ 0.6311,  1.0126, -0.9876,  0.1799,  0.0110]], requires_grad=True)


In [ ]:
print(target)

tensor([2, 4, 1])


In [ ]:
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch

discriminator = ElectraForPreTraining.from_pretrained("google/electra-base-discriminator")
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator")

sentence = "The quick brown fox jumps over the lazy dog"
fake_sentence = "that fox sky over the lazy dog"

fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode(fake_sentence, return_tensors="pt")
discriminator_outputs = discriminator(fake_inputs)
predictions = torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2)

[print("%7s" % token, end="") for token in fake_tokens]

# [print("%7s" % int(prediction), end="") for prediction in predictions.tolist()]

   that    fox    sky   over    the   lazy    dog

[None, None, None, None, None, None, None]

In [ ]:
print(predictions)

tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0.]], grad_fn=<RoundBackward0>)


In [ ]:
print(discriminator_outputs[0])

tensor([[-3.2616, -4.4307,  0.0990, -1.4849, -4.9597, -4.3168, -4.6388, -2.3057,
         -3.2615]], grad_fn=<SqueezeBackward1>)


In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("sultan/BioM-ELECTRA-Base-Discriminator")

model = AutoModelForPreTraining.from_pretrained("sultan/BioM-ELECTRA-Base-Discriminator")

fake_sentence = "I will watch a movie tonight"

fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode(fake_sentence, return_tensors="pt")
discriminator_outputs = discriminator(fake_inputs)
predictions = torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2)

[print("%7s" % token, end="") for token in fake_tokens]

[print("%7s" % prediction, end="") for prediction in predictions.tolist()]


      i   will    wat   ##ch      a    mov   ##ie    ton ##ight[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

[None]

In [ ]:
print(predictions )
print("\n")
discriminator_outputs

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],
       grad_fn=<RoundBackward0>)




ElectraForPreTrainingOutput([('logits',
                              tensor([[-5.4027, -2.7329, -3.3122, -1.2622,  1.0583, -3.9619, -3.4206, -3.1851,
                                       -4.1895, -4.0203, -4.3361]], grad_fn=<SqueezeBackward1>))])